In [1]:
import scipy.stats as st
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [51]:
def sigm(x):
    return 1 / (1 + np.exp(-x))

In [19]:
def dist(points, p1, p2=None, ):
    if p2 is None:
        # In case we give 1 point, it must be the actual coordinates
        return np.sum((points - p1) ** 2, axis=1)
    else:
        # When 2 points are given, they are the indices
        return np.sum((points[p1] - points[p2]) ** 2)

In [3]:
def in_ring(points, center, diameter, eps=None):
    
    # if eps is None, we check for the whole circle
    dist_center = dist(points, points[center])
    in_ = dist_center <= diameter
    if eps is None:
        return in_
    in_eps = dist_center >= diameter - eps
    return np.logical_and(in_, in_eps)

In [4]:
"""
NOT IN USE YET, MIGHT BE USELESS !!!
"""

def points_in_area(points, supp_1, supp_2, city_range, eps=None):
    diameter = dist(points, supp_1, supp_2)

    # Check which points are in the whole circle, but not in the smaller one
    in_s1_area = in_ring(points, supp_1, diameter, eps)
    in_s2_area = in_ring(points, supp_2, diameter, eps)
    # Points must lie in rings of both circles to be candidates
    valid_points = np.logical_and(in_s1_area, in_s2_area)
    # Do the same for the second circle
    return city_range[valid_points]

In [167]:
%%timeit
s1 = np.random.randint(0, 50_000)
s2 = np.random.randint(0, 50_000)
points_in_eps_area(points, s1, s2, 1).shape

2.63 ms ± 717 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
def cost(diameter, lambda_, n):
    return lambda_ * n * np.pi * diameter / 4

In [81]:
def cost_diff(sel_city, state, supp_1, supp_2, points, rewards, lambda_, city_range):
    # Check if we want to remove or add the city
    want_add = state[sel_city] == 0
    
    if want_add:
        # if we have less than 2 points -> one of the supports might be None
        if supp_1 is None and supp_2 is None:
            return rewards[sel_city], sel_city, None
        elif supp_2 is None:
            return rewards[sel_city] - cost(dist(points, supp_1, sel_city), lambda_, n), supp_1, sel_city
        # We make sure that if one of the support is None, it is the second one
        
        curr_cities = city_range[state == 1]
        all_d_to_new = dist(points[state == 1, :], points[sel_city])
        #dist_to_points_in = all_d_to_new[curr_cities, :]
        max_dist = np.max(all_d_to_new)
        diameter = dist(points, supp_1, supp_2)
        # If max distance with new point is the same as current diameter, support points remain. Otherwise, new points is a support
        if diameter >= max_dist:
            return rewards[sel_city], supp_1, supp_2
        else:
            old_cost = cost(diameter, lambda_, len(city_range))
            new_cost = cost(max_dist, lambda_, len(city_range))
            # Must compute new support points. Note that dist_to_points_in only cotains distance to points in area. However we want the global index, which is in array pia
            m = np.argmax(all_d_to_new)
            new_supp_2 = curr_cities[m]
            return rewards[sel_city] - new_cost + old_cost, sel_city, new_supp_2
    # When we remove a state,
    else:
        # if we don't remove a support point, then we only decrease score and keep support
        if sel_city not in (supp_1, supp_2):
            return - rewards[sel_city], supp_1, supp_2
        else:
            # If we have too few points, we might remove the support points -> care with that
            if sum(state) == 2:
                # Keep one supp point only -> no more cost, only the reward of the remaining state
                removed_supp = supp_1 if supp_1 == sel_city else supp_2
                remain_supp = supp_2 if supp_1 == sel_city else supp_1
                return - rewards[removed_supp] + cost(dist(points, supp_1, supp_2), lambda_, n), remain_supp, None
            elif sum(state) == 1:
                return - rewards[supp_1], None, None
            # Otherwise, we have more that 2 states remaining -> we are fine
            
            # Must find a new support, and keep one of the old ones
            rem_supp = supp_1 if sel_city == supp_2 else supp_2
            curr_cities = city_range[state == 1]
            
            # Change the state so as to compute the max. distance if selected state was removed
            state[sel_city] = 0
            new_cities = city_range[state == 1]
            all_d_to_remaining = dist(points[state == 1, :], points[rem_supp])
            new_supp = np.argmax(all_d_to_remaining)
            new_supp = new_cities[new_supp]
            # Restore state: we only wanted to compute the score diff, not make a move
            state[sel_city] = 1
            
            old_cost = cost(dist(points, supp_1, supp_2), lambda_, len(city_range))
            new_cost = cost(dist(points, rem_supp, new_supp), lambda_, len(city_range))
            
            return -rewards[sel_city] - new_cost + old_cost, rem_supp, new_supp

In [31]:
def flip(x):
    return 1 - x # = 0 when x == 1 and 1 when x == 0

In [91]:
def is_move_accepted(sel_city, state, supp_1, supp_2, points, rewards, lambda_, city_range, unif_rv, beta):
    delta_f, nsupp_1, nsupp_2 = cost_diff(sel_city, state, supp_1, supp_2, points, rewards, lambda_, city_range)
    
    # Note: we have beta here instead of - beta because we maximize the function, as opposed to the Ising model where we minimize it.
    #accept_prob: sigmoid of the cost change
    accept_prob = 1/2 * ( 1 + np.tanh(beta * delta_f))
    print(accept_prob)
    return unif_rv <= accept_prob, nsupp_1, nsupp_2

In [53]:
def rw_step(state, supp_1, supp_2, points, rewards, lambda_, city_range, beta):
    # Generate random quantities required to work
    sel_city = np.random.choice(city_range)
    unif_rv = np.random.uniform()
    # Check if proposed move is accepted, and what would the new support points be
    is_ma, nsupp_1, nsupp_2 = is_move_accepted(sel_city, state, supp_1, supp_2, points, rewards, lambda_, city_range, unif_rv, beta)
    # if move accepted, modify the state
    if is_ma:
        
        state[sel_city] = flip(state[sel_city])
        return nsupp_1, nsupp_2
    else:
        # If rejected, just return the current support
        return supp_1, supp_2

In [54]:
def all_pairs_sq_dist(points):
    n_points = len(points)
    p1 = points[np.newaxis, :, :]
    p2 = points[:, np.newaxis, :]
    res = np.sum((p1 * p2) ** 2, axis=2)
    max_index = np.argmax(res)
    row = max_index // n_points
    col = max_index % n_points
    return res, row, col

In [88]:
def optimize(points, rewards, lambda_, initial_nb_states=5, burning_steps=50_000, seed=999):
    np.random.seed(seed)
    N = points.shape[0]
    city_range = np.arange(N)
    initial_cities = np.random.permutation(city_range)[:initial_nb_states]
    # Create state representation & signal that cities are selected
    state = np.zeros(N)
    state[initial_cities] = 1
    selected_points = points[initial_cities, :]
    curr_max_dist, supp_1, supp_2 = all_pairs_sq_dist(selected_points)
    # supp_1 and supp_2 represent the maxs in the subset, we need to know what there indice in the whole is
    supp_1 = initial_cities[supp_1]
    supp_2 = initial_cities[supp_2]
    
    # Initialize the different betas used to simulate the chain
    # np.linspace(0.01, 3, num=5)
    betas = [0.01]
    # Run the simulation a certain number of steps and return the final state
    for beta in betas:
        for _ in range(burning_steps):
            supp_1, supp_2 = rw_step(state, supp_1, supp_2, points, rewards, lambda_, city_range, beta)
            
    return state
        

In [92]:
points = np.random.uniform(0, 3, size=(300, 2))
rewards = np.random.uniform(0, 1, size=300)

In [93]:
%%time
res = optimize(points, rewards, 0)

0.50404979284022
6.0
0.5029656324343893
6.0
0.5034026550764785
6.0
0.5048982477075835
6.0
0.5014224617550521
6.0
0.5039296289983839
7.0
0.503212153473835
8.0
0.5009569470345205
9.0
0.5025617839100107
10.0
0.5049159992531127
10.0
0.5030418700696779
11.0
0.5036784199583431
12.0
0.5009506773640124
12.0
0.5024444624219464
13.0
0.5018284583400112
13.0
0.5041615819525076
13.0
0.5023588382657825
14.0
0.5004659845902761
15.0
0.504812759938004
16.0
0.5013273810605564
16.0
0.5037284632288779
16.0
0.5024072637032113
17.0
0.5049305029272279
17.0
0.4960703710016161
17.0
0.5034026550764785
18.0
0.5045211746250621
18.0
0.5032660326267657
19.0
0.5042592894807159
19.0
0.5027285026128125
20.0
0.501495667972588
21.0
0.5000453275729403
22.0
0.5034222264643352
23.0
0.5023776736388568
24.0
0.5034970225672165
25.0
0.5049622184240804
26.0
0.5011448705136494
26.0
0.4976223263611433
26.0
0.5017279514606423
26.0
0.5048011604897198
26.0
0.5039518117098339
26.0
0.5005153793971163
26.0
0.504171249688568
27.0
0.5000

In [83]:
sum(res)

137.0